In [3]:
from PIL import Image
import pytesseract as tess


In [4]:
im=Image.open("test_image\\test1.jpeg")

In [5]:
print(im)

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x771 at 0x27EA914C690>


In [6]:
image_data=tess.image_to_string(im)
print(image_data)

Independent
4¢ | University,
s° Bangladesh

Sourav Kumar Saha
Student
1D: 1930188

dK

wut LAL



#using open ai vision model to get the text form the image


In [7]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import base64
load_dotenv()

True

In [8]:
# OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
client=OpenAI()

In [9]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
    
image_path = "test_image\\test1.jpeg"
base64_image = encode_image(image_path)

In [10]:
responce=client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[ 
        {
            "role":"system",
            "content":"You are a helpful assistant."
        },
        {
            "role":"user",
            "content":[
                {
                    "type":"text",
                    "text":"""extract text from image of id card. Text like university name, student name,
                    student id. give me the extracted information as dict format key value piar.
                    example:
                    [
                        {
                            University: university name,
                            Student Name: student name, 
                            ID: student id,
                        },
                        {
                            University: university name,
                            Student Name: student name, 
                            ID: student id,
                        },
                    
                    ]
                    
                    if the  there is multiple id there stracture data in list of dictionary format as given .
                    """
                },
                {
                    "type":"image_url",
                    "image_url":{"url": f"data:image/jpeg;base64,{base64_image}"}
                }
            ]
        }
    ],
    max_tokens=300,
)

In [11]:
print(responce)


ChatCompletion(id='chatcmpl-Ai7UNyCdxO7tIiICb8csPgYgJNfWc', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n[\n    {\n        "University": "Independent University, Bangladesh",\n        "Student Name": "Sourav Kumar Saha",\n        "ID": "1930188"\n    }\n]\n```', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1735078583, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_d02d531b47', usage=CompletionUsage(completion_tokens=43, prompt_tokens=719, total_tokens=762, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [12]:
responce.choices[0].message.content


'```json\n[\n    {\n        "University": "Independent University, Bangladesh",\n        "Student Name": "Sourav Kumar Saha",\n        "ID": "1930188"\n    }\n]\n```'

In [13]:
responce_data=responce.choices[0].message.content


In [1]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain_openai import OpenAI


In [2]:
llm =OpenAI(temperature=0,model="gpt-4o-mini")

In [ ]:
# ectract_image_data_prompt=PromptTemplate(
#     input_variables=["image_base64"],
    
#     template="""
#         you are an ai assistant. you have to extract text from image of id card. Text like university name, student name,
#         student id. universty establis year. give the extracted impormation in list of dictionary format.
#         example:
#         [
#            {{
#                 University: university name,
#                 Student Name: student name, 
#                 ID: student id,
#             }},
#             {{
#                 University: university name,
#                 Student Name: student name, 
#                 ID: student id,
#             }},
                    
#         ]
                    
#         if the  there is multiple id there stracture data in list of dictionary format as given .
        
    
    
#     """  
# )

# extract_image_data_chain=LLMChain(llm=llm,prompt=ectract_image_data_prompt)


In [27]:
# extract_image_data_chain.invoke({"image_base64":base64_image})

NotFoundError: Error code: 404 - {'error': {'message': 'This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}